In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#Get US National data for 2000_2009 API data
url_us_county_pop = "https://api.census.gov/data/2010/dec/sf1?get=P012001,P012002,P012026,NAME&for=county:*&in=state:*&for=us:1"
us_county_pop_response = requests.get(url_us_county_pop)
us_county_pop_data = us_county_pop_response.json()



In [61]:
# function cleanse series and return a list

def cleanse_data(top_pop_list):
    cleaned_pop_list = []
    for item in list(top_pop_list):
        pop_value = item
        if (pop_value.find("(") >=0):     
            end_position = pop_value.find("(") 
        else: 
            end_position = len(pop_value)
        #print(end_position)
        #print(pop_value[0:end_position])
        cleaned_pop_list.append(pop_value[0:end_position])
    return cleaned_pop_list



In [62]:
# Add it dataframe and conver
df_us_county_pop = pd.DataFrame(us_county_pop_data)
### Filter out the header row
df_us_county_pop = pd.DataFrame(df_us_county_pop[1:])
df_us_county_pop_final = df_us_county_pop.rename(columns = {0:"tot_county_pop", 1:"tot_county_male_pop", 2:"tot_county_female_pop",\
                                                            3:"county_name_state", 4:"state_code", 5:"county_code"})
df_us_county_pop_final.head()
#df_us_county_pop.count()
#df_us_county_pop.dtypes
#df_us_2000_2009_data = pd.DataFrame(us_2000_2009_data[1:], columns=us_2000_2009_data[0]).rename(columns = {"DATE_DESC":"PERIOD_DESC", "DATE":"PERIOD", "us":"US"})


,tot_county_pop,tot_county_male_pop,tot_county_female_pop,county_name_state,state_code,county_code
1,54571,26569,28002,"Autauga County, Alabama",01,001
2,22915,12301,10614,"Bibb County, Alabama",01,007
3,34215,16363,17852,"Chambers County, Alabama",01,017
4,25989,12888,13101,"Cherokee County, Alabama",01,019
5,25833,12214,13619,"Clarke County, Alabama",01,025


In [68]:
## create a dataframe from CSV file
csv_file = "Total_us_pop_2010_per_state.csv"
df_state_pop = pd.read_csv(csv_file, encoding="ISO-8859-1" )
### Filter out the header row
df_state_pop = pd.DataFrame(df_state_pop[1:])

## Get the subset of columns
df_state_pop_final = df_state_pop[["GEO.id2","SUBHD0101_S01", "SUBHD0102_S01", "SUBHD0103_S01"]]\
    .rename(columns = {"GEO.id2":"state_code", "SUBHD0101_S01":"total_pop", "SUBHD0102_S01":"total_male_pop", "SUBHD0103_S01":"total_female_pop"})
    
#cleanse the total pop data which has paranthesis
cleaned_total_pop = cleanse_data(df_state_pop_final ["total_pop"])

df_state_pop_final["cleaned_total_pop"] = cleaned_total_pop

df_state_pop_final.head()

,state_code,total_pop,total_male_pop,total_female_pop,cleaned_total_pop
1,01,4779736(r38235),2320188,2459548,4779736
2,02,710231(r38823),369628,340603,710231
3,04,6392017,3175823,3216194,6392017
4,05,2915918(r39193),1431637,1484281,2915918
5,06,37253956,18517830,18736126,37253956


In [136]:
df = pd.merge(df_state_pop_final, df_us_county_pop_final, on="state_code", how="inner")

#df.head()

df = df[["county_name_state", "county_code", "state_code", "cleaned_total_pop", "total_male_pop", "total_female_pop",\
        "tot_county_pop", "tot_county_male_pop", "tot_county_female_pop"]]

df["tot_county_pop"] = pd.to_numeric(df["tot_county_pop"])

pct_tot_pop_cty_st = (pd.to_numeric(df["tot_county_pop"])/pd.to_numeric(df["cleaned_total_pop"]))*100
df["pct_tot_pop_cty_st"]=pct_tot_pop_cty_st

pct_tot_male_pop_cty_st = (pd.to_numeric(df["tot_county_male_pop"])/pd.to_numeric(df["total_male_pop"])) * 100
df["pct_tot_male_pop_cty_st"]=pct_tot_male_pop_cty_st

pct_tot_female_pop_cty_st = (pd.to_numeric(df["tot_county_female_pop"])/pd.to_numeric(df["total_female_pop"])) * 100
df["pct_tot_female_pop_cty_st"]=pct_tot_female_pop_cty_st


#df.head()

df_final = df.sort_values(["state_code", "tot_county_pop"], ascending=[True, False]).reset_index(drop=True)



df_final.head(100)


,county_name_state,county_code,state_code,cleaned_total_pop,total_male_pop,total_female_pop,tot_county_pop,tot_county_male_pop,tot_county_female_pop,pct_tot_pop_cty_st,pct_tot_male_pop_cty_st,pct_tot_female_pop_cty_st
0,"Jefferson County, Alabama",073,01,4779736,2320188,2459548,658466,311813,346653,13.776200,13.439126,14.094175
1,"Mobile County, Alabama",097,01,4779736,2320188,2459548,412992,198374,214618,8.640477,8.549911,8.725912
2,"Madison County, Alabama",089,01,4779736,2320188,2459548,334811,164320,170491,7.004801,7.082185,6.931802
3,"Montgomery County, Alabama",101,01,4779736,2320188,2459548,229363,109167,120196,4.798654,4.705093,4.886914
4,"Shelby County, Alabama",117,01,4779736,2320188,2459548,195085,95557,99528,4.081502,4.118502,4.046597
5,"Tuscaloosa County, Alabama",125,01,4779736,2320188,2459548,194656,94400,100256,4.072526,4.068636,4.076196
6,"Baldwin County, Alabama",003,01,4779736,2320188,2459548,182265,89196,93069,3.813286,3.844344,3.783988
7,"Lee County, Alabama",081,01,4779736,2320188,2459548,140247,69126,71121,2.934200,2.979328,2.891629
8,"Morgan County, Alabama",103,01,4779736,2320188,2459548,119490,58886,60604,2.499929,2.537984,2.464030
9,"Calhoun County, Alabama",015,01,4779736,2320188,2459548,118572,57176,61396,2.480723,2.464283,2.496231


In [137]:
## loading into Mysql
# PyMySQL 
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [138]:
engine = create_engine("mysql://test@localhost:3306/favorite_db")
conn = engine.connect()

In [140]:
df_final.to_sql("us_county_population", conn)
#data.head()

In [142]:
## exporting in to Excel
df_final.to_excel("us_county_population.xls")